# Householder Matrix

A householder matrix for a given $v\in\mathbb{R}^n$ is
$$
    H_v = I - 2\frac{vv^T}{v^Tv}
$$

**Theorem** Given any vector $x\in\mathbb{R}^n$, there is a vector $v\in\mathbb{R}^n$ s.t.
$$
    H_v(x) = x-2\frac{vv^Tx}{v^Tv} = (\lambda, 0, \cdots, 0)
$$
i.e. all but first coordinate is non zero

---

Now given a matrix
$$\left(
    \begin{matrix}
    \alpha & b^T \\
    b & C
    \end{matrix}\right)
$$

We can calculate a $H_{n,n-1}$ s.t. $Hb\in\varepsilon ^{n-1}_1$
Now we continue recursively

In [3]:
using LinearAlgebra;
function Hᵥ(v)
    n = size(v, 1);
    I -2(v*v')/(v'v)
end

Hᵥ (generic function with 1 method)

In [4]:
function Hxe(x, e)
    β = e'x;
    if β == 0
        c = √(x'x)
    else
        c = sign(β)*√(x'x)
    end
    
    return Hᵥ(x+c*e)
end

Hxe (generic function with 1 method)

In [11]:
function householder_tridiagonlize(A)
    # We have taken the matrix to be of the form
    # [α bᵀ]
    # [b C ]
    
    # Defining the algorithm recursively
    n = size(A, 1);   # Size of the matrix
    if n==2
        # if n is 2, then matrix is already tridiagonalized
        return A, 1. .*Array(I, 2, 2);
    end
    
    # work with first one
    b = A[2:end, 1];   # Vector b
    e = zeros(size(b)); e[1, 1] = 1;
    C = A[2:end, 2:end];
    
    # Gives us the Householder matrix to multiply
    Hn_1 = Hxe(b, e);
    D    = Hn_1'*C*Hn_1;
    d    = Hn_1*b;
    Hn = 1. .* Matrix(I, n, n);
    Hn[2:end, 2:end] = Hn_1;
    
    # Previous Householder matrix
    DD, Hn_2 = householder_tridiagonlize(D);  # Diagonalised D and matrix used
    Hn_2_ext   = 1. .* Array(I, n, n);    # Start with old Hn_1 and make new one
    Hn_2_ext[2:end, 2:end] = Hn_2;        # Adjust it to size n
    
    
    # Now work with current matrix
    H = Hn*Hn_2_ext;
    A[2:end, 1] = d;
    A[1, 2:end] = d';
    A[2:end, 2:end] = DD;
    return A, H;
end

householder_tridiagonlize (generic function with 1 method)

In [12]:
AA = rand(5, 5);
A = AA+AA';
display(A);
T, Q = householder_tridiagonlize(A);
display(T)

5×5 Array{Float64,2}:
 1.69364   1.00461   0.906304   1.25918   0.113917
 1.00461   1.70888   1.19435    0.597512  0.413359
 0.906304  1.19435   0.0712916  0.588628  1.36575
 1.25918   0.597512  0.588628   0.925937  1.57734
 0.113917  0.413359  1.36575    1.57734   1.1419

5×5 Array{Float64,2}:
  1.69364      -1.8518        0.0        8.32667e-17   0.0
 -1.8518        2.66111      -1.99825    1.11022e-16  -3.33067e-16
  0.0          -1.99825       0.392891  -0.799317      0.0
  8.32667e-17   1.11022e-16  -0.799317   0.363879      0.888197
  0.0          -3.33067e-16   0.0        0.888197      0.430128

In [13]:
T[abs.(T) .< 1e-15] .= 0
display(T)

5×5 Array{Float64,2}:
  1.69364  -1.8518    0.0        0.0       0.0
 -1.8518    2.66111  -1.99825    0.0       0.0
  0.0      -1.99825   0.392891  -0.799317  0.0
  0.0       0.0      -0.799317   0.363879  0.888197
  0.0       0.0       0.0        0.888197  0.430128

In [15]:
Q'*A*Q

5×5 Array{Float64,2}:
  1.69364    1.00461   -0.463048  -1.36544    -0.58402
  1.00461   -0.471648  -0.825132  -1.05035     0.822044
 -0.463048  -0.825132   0.216067   0.638963   -0.545686
 -1.36544   -1.05035    0.638963   3.34764    -0.0888681
 -0.58402    0.822044  -0.545686  -0.0888681   0.755944